In [1]:
import pandas as pd
import numpy as np

dropbox_path = r'C:\Users\ed\Dropbox\Ed\Ed Uni work\EME\Data\Original Data\Large_Index_Data.xlsx'

In [2]:
index_df = pd.read_excel( dropbox_path, date_parser=True)

In [3]:
index_df.columns

Index(['Name', 'S&P 500 COMPOSITE - PRICE INDEX', 'Name.1',
       'S&P 500 COMPOSITE - PRICE INDEX (~£ )', 'Name.2',
       'US $ TO UK £ (WMR) - EXCHANGE RATE', 'Name.3',
       'FTSE ALL SHARE - PRICE INDEX', 'MSCI UK - PRICE INDEX',
       'FT 30 ORDINARY SHARE - PRICE INDEX', 'FTSE 100 - PRICE INDEX',
       'Name.4', 'DAX 30 PERFORMANCE - PRICE INDEX (~£ )', 'Name.5',
       'MSCI AUSTRALIA - PRICE INDEX (~£ )', 'Name.6',
       'ISRAEL TA 125 - PRICE INDEX (~£ )', 'FTSE ISRAEL - PRICE INDEX (~£ )',
       'Name.7', 'ISRAEL TA 125 - PRICE INDEX (~U$)',
       'FTSE ISRAEL - PRICE INDEX (~U$)', 'Name.8',
       'NASDAQ COMPOSITE - PRICE INDEX', 'NASDAQ 100 - PRICE INDEX', 'Name.9',
       'UK £ TO US $ (WMR) - EXCHANGE RATE'],
      dtype='object')

In [4]:
# ## S&P500 Composite in GBP and FTSE100 in GBP ##


# SP500_df1 = index_df[['Name.1', 'S&P 500 COMPOSITE - PRICE INDEX (~£ )']].copy()
# FTSE100_df1 = index_df[['Name.3', 'FTSE 100 - PRICE INDEX']].copy()
# GBPUSD_df1 = index_df[['Name.2','US $ TO UK £ (WMR) - EXCHANGE RATE']].copy()
# FTSE_ALL_df1 = index_df[['Name.3', 'FTSE ALL SHARE - PRICE INDEX']].copy()
# MSCI_UK_df1 = index_df['Name.3','MSCI UK - PRICE INDEX' ].copy()
# FT30_df1

# #Dropping the first two rows and parsing the date for each index
# SP500_df1 = SP500_df1.loc[2:, :]
# FTSE100_df1 = FTSE100_df1.loc[2:, :]
# SP500_df1['Date'] = pd.to_datetime(SP500_df1['Name'], format = '%Y/%m/%d')
# FTSE100_df1['Date'] = pd.to_datetime(FTSE100_df1['Name.3'], format = '%Y/%m/%d')

# #Dropping the Name column
# SP500_df1 = SP500_df1.drop('Name', axis = 1)
# FTSE100_df1 = FTSE100_df1.drop('Name.3', axis = 1)

# #Removing observations that are NaN. (That is, observations from before the FTSE existed that are in the FTSE index)
# FTSE100_df1 = FTSE100_df1.dropna()
# SP500_df1 = SP500_df1.dropna()


# #Performing an 'inner' merge, this means we'll lose observations for the longer dataset. We'll also lose any observations that
# #occur in one index but not another - I think this shouldn't be a problem as both indices should just be jointly missing
# #weekends but need to check. TODO: Check date observations actually match.
# joint_index_1 = SP500_df1.merge(how='inner', right = FTSE100_df1, on = 'Date')

# #Rearranging column order
# joint_index_1 = joint_index_1[['Date','S&P 500 COMPOSITE - PRICE INDEX (~£ )','FTSE 100 - PRICE INDEX' ]]
# joint_index_1.head()

# #Saving to Excel
# #joint_index_1.to_excel(r'E:\EME Data\Clean Data\Indices\SP500 FTSE100 GBP.xlsx')

In [5]:
def index_grabber(data, name, index):
    '''
    Cleaning the excel spreadsheet by ensuring each date matches the correct index and collating into one dataframe
    
    '''
    # finding specific index
    dataframe = data[[ name, index]].copy()
    
    # Dropping first two rows as they aren't actual observations but titles and parsing dates
    dataframe = dataframe.loc[2:, :]
    dataframe['Date'] = pd.to_datetime(dataframe[name], format = '%Y/%m/%d')
    
    # Removing the now irrelevant name column and dropping any empty rows at the bottom of the dataframe from mismatched index
    # length
    dataframe = dataframe.drop( name, axis = 1)
    dataframe = dataframe.loc[dataframe['Date'].notnull()]
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def indice_merger(*args):
    
    '''
    First create a daterange from the very earliest date in the indices provided to the last. Then merge all the indices
    to this date dataframe
       
    '''
    first_dates = []
    last_dates = []
    for arg in args:
        first_date = arg.loc[0, 'Date']
        first_dates = first_dates + [first_date]
        
        last_date = arg.loc[ len(arg['Date']) - 1, 'Date']
        last_dates = last_dates + [last_date]

    date_range = pd.date_range( start = min(first_dates), end = max(last_dates), frequency = 'D')
    # print(min(first_dates), max(last_dates))
    joint_df = pd.DataFrame(date_range, columns = ['Date'])
    
    for arg in args:
        joint_df = joint_df.merge(how = 'left', right = arg, on = 'Date')
    return joint_df
    

In [6]:
FTSE100 = index_grabber(index_df,index  = 'FTSE 100 - PRICE INDEX', name = 'Name.3' )
FTSEALL = index_grabber(index_df, index = 'FTSE ALL SHARE - PRICE INDEX', name = 'Name.3')
SP500 = index_grabber(index_df, index ='S&P 500 COMPOSITE - PRICE INDEX (~£ )', name ='Name.1'  )
MSCI_UK = index_grabber(index_df, index = 'MSCI UK - PRICE INDEX', name = 'Name.3')
FT30 = index_grabber(index_df, index ='FT 30 ORDINARY SHARE - PRICE INDEX', name = 'Name.3' )
DAX30 = index_grabber(index_df, index ='DAX 30 PERFORMANCE - PRICE INDEX (~£ )', name = 'Name.4')
MSCI_AUS = index_grabber(index_df, index ='MSCI AUSTRALIA - PRICE INDEX (~£ )', name = 'Name.5' )
ISRAEL_TA_GBP = index_grabber(index_df, index ='ISRAEL TA 125 - PRICE INDEX (~£ )', name ='Name.6' )
ISRAEL_FTSE_GBP = index_grabber(index_df, index = 'FTSE ISRAEL - PRICE INDEX (~£ )', name = 'Name.6')
NASDAQ_COMP = index_grabber(index_df, index ='NASDAQ COMPOSITE - PRICE INDEX', name = 'Name.8' )
NASDAQ_100 = index_grabber(index_df, index = 'NASDAQ 100 - PRICE INDEX', name = 'Name.8')
GBP_USD = index_grabber(index_df, index ='UK £ TO US $ (WMR) - EXCHANGE RATE', name = 'Name.9' )


In [7]:
merged = indice_merger(FTSE100,
                       FTSEALL,
                       MSCI_UK,
                       FT30,
                       DAX30,
                       MSCI_AUS,
                       ISRAEL_TA_GBP,
                       ISRAEL_FTSE_GBP,
                       NASDAQ_COMP,
                       NASDAQ_100,
                       GBP_USD)
merged

,Date,FTSE 100 - PRICE INDEX,FTSE ALL SHARE - PRICE INDEX,MSCI UK - PRICE INDEX,FT 30 ORDINARY SHARE - PRICE INDEX,DAX 30 PERFORMANCE - PRICE INDEX (~£ ),MSCI AUSTRALIA - PRICE INDEX (~£ ),ISRAEL TA 125 - PRICE INDEX (~£ ),FTSE ISRAEL - PRICE INDEX (~£ ),NASDAQ COMPOSITE - PRICE INDEX,NASDAQ 100 - PRICE INDEX,UK £ TO US $ (WMR) - EXCHANGE RATE
0,1957-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
1,1957-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
2,1957-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1957-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1957-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
5,1957-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
6,1957-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
7,1957-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
8,1957-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
9,1957-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:

# merged2 = merged
# merged2.to_csv(r'C:\Users\ed\Dropbox\Ed\Ed Uni work\EME\Data\Clean Data\Indices\All_indices_cleaned_test.csv')
# merged2

,Date,FTSE 100 - PRICE INDEX,FTSE ALL SHARE - PRICE INDEX,MSCI UK - PRICE INDEX,FT 30 ORDINARY SHARE - PRICE INDEX,DAX 30 PERFORMANCE - PRICE INDEX (~£ ),MSCI AUSTRALIA - PRICE INDEX (~£ ),ISRAEL TA 125 - PRICE INDEX (~£ ),FTSE ISRAEL - PRICE INDEX (~£ ),NASDAQ COMPOSITE - PRICE INDEX,NASDAQ 100 - PRICE INDEX,UK £ TO US $ (WMR) - EXCHANGE RATE
0,1957-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
1,1957-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
2,1957-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1957-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1957-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
5,1957-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
6,1957-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
7,1957-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
8,1957-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35737
9,1957-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# merged.to_excel(r'C:\Users\ed\Dropbox\Ed\Ed Uni work\EME\Data\Clean Data\Indices\All_indices_cleaned.xls')